In [1]:
# 공통
import pandas as pd
import numpy as np
import statsmodels.api as sm
from scipy.stats import norm
from sklearn.metrics import mean_squared_error

In [14]:
# Train 데이터 불러오기 (00년 ~ 21년)
# 2021년 Train / 2022년 2022Train

data = pd.read_csv('HTrain.csv')
data = data[38:]
# data = data.drop(['total'], axis = 1)

# data = data.drop(['china'],axis=1)
# data = data.drop(['usa'], axis = 1)
# data = data.drop(['vietnam'], axis = 1)
# data = data.drop(['hongkong'], axis = 1)
# data = data.drop(['japan'], axis = 1)

# data = data.drop(['korea_index'], axis = 1)
data = data.drop(['oecd_index'], axis = 1)
data = data.drop(['ex_rate'], axis = 1)
# data = data.drop(['oil'], axis = 1)
    
data = data.set_index('date')
data.head()

,total,china,usa,vietnam,hongkong,japan,korea_index,oil
date,,,,,,,,
2003-03-01,15378850237,2659292622,2749138408,252511469,1072609239,1371507580,98.53373,33.16
2003-04-01,15720592277,2560138383,2807774807,254067765,1138131661,1406227585,98.28757,28.14
2003-05-01,14676091579,2475020616,2633439848,236404469,1119015400,1350886060,98.22107,28.07
2003-06-01,15656327385,2609510640,2803208194,228234538,1154056078,1319147409,98.32925,30.52
2003-07-01,15431714254,2964910238,2497304252,194171150,1103554595,1382143845,98.57484,30.70


In [15]:
# 2021년 예측

train = data[:'2021-06-01']['total']

exog1 = data.drop('total', axis=1)[:'2021-06-01']
exog1 = sm.add_constant(exog1)

exog2 = data.drop('total', axis=1)['2021-07-01':]
exog2 = sm.add_constant(exog2)


In [81]:
# # 2022년 예측

# train = data[:'2021-12-01']['japan']

# exog1 = data.drop('japan', axis=1)[:'2021-12-01']

# exog2 = data.drop('japan', axis=1)['2022-01-01':]

In [16]:
def predict(exog1, exog2):

    mod = sm.tsa.statespace.SARIMAX(train, exog = exog1, order = (1,1,0), seasonal_order = (1, 1, 1, 12))
    fit_res = mod.fit(disp = False)

    forecast = fit_res.forecast(steps = 6, exog = exog2)

    # 2021년 실제값
    actual = data['total'][-6:]

    # # 2022년 실제값
    # actual = [2619529000, 2480632000, 2778454000, 2637731000, 2831080000]

    rmse = np.sqrt(mean_squared_error(actual, forecast))
    
    return rmse

# , seasonal_order = (0, 1, 1 , 12)

In [17]:
predict(exog1, exog2)

c:\Users\ASIA-08\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\ASIA-08\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


914333331.4593843

In [10]:
# mod = sm.tsa.statespace.SARIMAX(train, exog = exog1, order = (1,1,1), seasonal_order = (0, 1, 1 , 12))
# fit_res = mod.fit(disp = False)

# forecast = fit_res.forecast(steps = 6, exog = exog2)

# forecast

c:\Users\ASIA-08\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\ASIA-08\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


2021-07-01    5.521436e+10
2021-08-01    5.326373e+10
2021-09-01    5.559890e+10
2021-10-01    5.438966e+10
2021-11-01    5.757340e+10
2021-12-01    5.995990e+10
Freq: MS, Name: predicted_mean, dtype: float64

In [143]:
# forecast.to_csv('koreaoil_2021.csv', header=['koreaoil_2021'], index=False)